In [1]:
import numpy as np
from numba import njit,prange
from scipy.optimize import differential_evolution
from functions import is_target_blocked, v_of_fy


In [2]:
delta_t = 0.01  # time step(s)

M1_start_pos = np.array([20000, 0, 2000])
FY1_start_pos = np.array([17800, 0, 0])
FY1_height = np.array([0, 0, 1800])  # the height of FY1(m)

Vm = 300 * -M1_start_pos / np.linalg.norm(M1_start_pos)  # velocity of missiles(m/s)

g = np.array([0, 0, -9.8])  # the gravity acceleration
V_cloud = np.array([0, 0, -3])  # the velocity of the cloud(m/s)




In [3]:
@njit(fastmath=True, parallel=True, cache=True, nogil=True)
def objective(x):
    theta, speed, t_11, t_12 = x
    V1 = v_of_fy(speed, theta)
    M1_end_pos = M1_start_pos + Vm * (t_11 + t_12)
    T1 = np.sqrt(np.sum(M1_end_pos ** 2)) / 300
    cloud_center = FY1_start_pos + FY1_height + V1 * (t_11 + t_12) + 0.5 * g * t_12**2
    counter = 0
    tmax = max(20.0, T1)
    steps = int(tmax / delta_t)
    for i in prange(steps):
        t = i * delta_t
        M1_pos_now = M1_end_pos + Vm * t
        cloud_center_now = cloud_center + V_cloud * t
        if is_target_blocked(M1_pos_now, cloud_center_now, t):
            counter += 1
    return -counter * delta_t

In [22]:
# pack the arguments into a single tuple for the objective function

bounds = [(0, 2*np.pi), (70, 140), (0, 5), (0, 5)]
result = differential_evolution(objective, bounds, popsize=8, disp=True, polish=True, maxiter=1000)
theta, speed, t_11, t_12 = result.x
max_time = -result.fun
print(f"最长遮蔽时间: {max_time:.3f} s")
print(f"最优参数: theta={theta:.3f}, speed={speed:.3f}, t_11={t_11:.3f}, t_12={t_12:.3f}")

differential_evolution step 1: f(x)= -2.48
differential_evolution step 2: f(x)= -2.48
differential_evolution step 3: f(x)= -2.48
differential_evolution step 4: f(x)= -2.48
differential_evolution step 5: f(x)= -2.48
differential_evolution step 6: f(x)= -2.48
differential_evolution step 7: f(x)= -2.48
differential_evolution step 8: f(x)= -2.48
differential_evolution step 9: f(x)= -2.48
differential_evolution step 10: f(x)= -3.75
differential_evolution step 11: f(x)= -3.75
differential_evolution step 12: f(x)= -3.75
differential_evolution step 13: f(x)= -3.75
differential_evolution step 14: f(x)= -4.09
differential_evolution step 15: f(x)= -4.09
differential_evolution step 16: f(x)= -4.09
differential_evolution step 17: f(x)= -4.11
differential_evolution step 18: f(x)= -4.11
differential_evolution step 19: f(x)= -4.11
differential_evolution step 20: f(x)= -4.11
differential_evolution step 21: f(x)= -4.11
differential_evolution step 22: f(x)= -4.2700000000000005
differential_evolution step

最长遮蔽时间: 4.610 s
最优参数: theta=0.104, speed=100.995, t_11=0.411, t_12=0.669
最长遮蔽时间: 4.610 s
最优参数: theta=0.092, speed=117.246, t_11=0.695, t_12=0.339
